In [4]:
# !pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

1. **Асинхронная фильтрация данных с использованием `yield` (0.5б)**

   Реализуйте генератор `async_filtered_numbers`, который принимает числа и фильтрует их по условию. Используйте `yield`, чтобы передать только те числа, которые соответствуют условию (например, четные).

In [5]:
async def async_filtered_numbers(numbers, filter_func):
    for i in numbers:
      if filter_func(i):
        yield i

async for number in async_filtered_numbers([1, 2, 3, 4, 5, 6], lambda x: x % 2 == 0):
    print(number)

2
4
6


2. **Асинхронное вычисление факториалов с ограничением по времени (1б)**

   Создайте корутину `async_factorial`, которая принимает целое число и возвращает его факториал. Используйте цикл для выполнения вычислений. На каждой итерации цикла добавьте небольшую задержку, чтобы дать другим задачам возможность выполняться, с помощью `await asyncio.sleep(0)`. Если выполнение займет больше 0.1 секунды, корутина должна прерывать вычисления и выбросить исключение `asyncio.TimeoutError`. Необходимо, чтобы сначала было напечатано успешно вычисленное значение (все значения), и только потом выброшено исключение.

In [6]:
import asyncio
import sys

sys.set_int_max_str_digits(40000)

async def async_factorial(n: int) -> int:
    result = 1
    start_time = asyncio.get_event_loop().time()

    for i in range(1, n + 1):
        result *= i
        await asyncio.sleep(0)

        elapsed_time = asyncio.get_event_loop().time() - start_time
        if elapsed_time > 0.1:
            print(f"Последний успешно вычисленный факториал: {result}")
            raise asyncio.TimeoutError(f"Операция прервана на {i}!")

    return result


async def main():
    try:
        print(await async_factorial(10000))
    except asyncio.TimeoutError as e:
        print(e)

    try:
        print(await async_factorial(5))
    except asyncio.TimeoutError as e:
        print(e)

asyncio.run(main())

Последний успешно вычисленный факториал: ...
Операция прервана на 9704!
120


 3. **Асинхронная обработка задач с управлением через `asyncio.wait` (1б)**

   Напишите корутину `task_handler`, которая принимает несколько задач и управляет их выполнением, используя `asyncio.wait`. Обратите внимание, что возвращает `asyncio.wait`. Пусть задачи завершаются только, когда будут выполнены три самые короткие по времени задачи. Остальные задачи должны остаться в состоянии ожидания.


In [7]:
async def task_handler(tasks):
    wrapped_tasks = [asyncio.create_task(task) for task in tasks]

    done, pending1 = await asyncio.wait(wrapped_tasks, return_when=asyncio.FIRST_COMPLETED)
    done, pending2 = await asyncio.wait(pending1, return_when=asyncio.FIRST_COMPLETED)
    done, pending3 = await asyncio.wait(pending2, return_when=asyncio.FIRST_COMPLETED)

async def sample_task(name, delay):
   await asyncio.sleep(delay)
   print(f"{name} завершена после {delay} секунд")

await task_handler([
   sample_task("Task1", 5),
   sample_task("Task2", 3),
   sample_task("Task3", 1),
   sample_task("Task4", 4),
   sample_task("Task5", 2)
])

Task3 завершена после 1 секунд
Task5 завершена после 2 секунд
Task2 завершена после 3 секунд


4. **Асинхронная очередь с приоритетом и таймаутом (2б)**

   Создайте асинхронную очередь с приоритетом. Пусть `producer` добавляет в очередь задачи с разным приоритетом, а `consumer` обрабатывает задачи в порядке приоритета. При этом задачи, которые находятся в очереди более 3 секунд, должны удаляться без обработки. Если очередь пуста в течение времени ожидания в 1 секунду, завершаем с сообщением "No tasks left to process." Это должно происходить в конце (потому что priority_producer не будет писать в конце). Убедитесь, что это сработает и в середине, использовав закомментированное `# await asyncio.sleep(3)`. Считайте, что `priority_consumer` требует 1.5 секунды на обработку одной задачи (вставьте `await asyncio.sleep(1.5)` после операции получения очередного запроса из очереди с учетом таймаута).


In [14]:
import asyncio
import time

async def priority_producer(queue):
    await queue.put((1, time.time(), "High Priority Task"))
    await asyncio.sleep(1)
    await queue.put((3, time.time(), "Low Priority Task"))
    await asyncio.sleep(1)
    #await asyncio.sleep(3)
    await queue.put((2, time.time(), "Medium Priority Task"))
    await asyncio.sleep(1)
    await queue.put((0, time.time(), "Very High Priority Task"))

async def priority_consumer(queue):
    while True:
        try:
            item = await asyncio.wait_for(queue.get(), timeout=1)
            priority, timestamp, task = item
            await asyncio.sleep(1.5)
            current_time = time.time()
            time_in_queue = current_time - timestamp
            if time_in_queue > 3:
                print(f"Task '{task}' was discarded due to timeout.")
                queue.task_done()
                continue
            else:
                print(f"Processing {task} with priority {priority}")
                queue.task_done()
        except:
            if queue.empty():
                print("No tasks left to process.")
                break

async def main():
    queue = asyncio.PriorityQueue()
    await asyncio.gather(priority_producer(queue), priority_consumer(queue))
asyncio.run(main())

Processing High Priority Task with priority 1
Processing Low Priority Task with priority 3
Processing Very High Priority Task with priority 0
Task 'Medium Priority Task' was discarded due to timeout.
No tasks left to process.


5. **Асинхронный объединитель данных из нескольких источников с фильтрацией (0.5б)**

   Реализуйте `merge_data_sources`, который объединяет данные из нескольких асинхронных источников. Используйте `yield from`, чтобы делегировать обработку данных. Добавьте фильтр, который будет удалять пустые данные.


In [22]:
from typing import AsyncGenerator, Callable
async def data_source_one():
   for data in ["Data1", None, "Data3", ""]:
     await asyncio.sleep(1)
     yield data

async def data_source_two():
   for data in ["Valid1", "", "Valid3"]:
     await asyncio.sleep(1)
     yield data

async def merge_data_sources(*sources: Callable[[], AsyncGenerator]):
    for source in sources:
        async for data in source():
            if data:
                yield data

async for data in merge_data_sources(data_source_one, data_source_two):
    print(data)

Data1
Data3
Valid1
Valid3


6. **Асинхронный агрегатор данных из API с несколькими типами запросов (3б)**

   Реализуйте корутину `api_aggregator`, которая отправляет асинхронные запросы к нескольким API. Пусть будут три типа запросов:
   - **Запросы типа "fast"** обрабатываются без задержки,
   - **Запросы типа "slow"** обрабатываются с задержкой 3 секунды,
   - **Запросы типа "unstable"** могут завершиться ошибкой с вероятностью 50%.

   `api_aggregator` должен запускать все запросы параллельно и обрабатывать результаты по мере их поступления. При ошибке запросов "unstable" должен происходить повторный запрос до трех раз. Запустите своё решение несколько раз и убедитесь, что все сценарии корректно отрабатывают

   **Пример использования:**
   ```python
   async def fast_request():
       return "Fast result"

   async def slow_request():
       await asyncio.sleep(3)
       return "Slow result"

   async def unstable_request():
       if random.random() > 0.5:
           raise ValueError("Unstable request failed")
       return "Unstable result"

   async def api_aggregator():
       pass  # Реализуйте агрегатор для запуска всех запросов и обработки результатов

   await api_aggregator()
   ```

In [32]:
import random

async def fast_request():
    return "Fast result"

async def slow_request():
    await asyncio.sleep(3)
    return "Slow result"

async def unstable_request():
    if random.random() > 0.5:
        raise ValueError("Unstable request failed")
    return "Unstable result"

async def handle_unstable_request():
    attempts = 3
    for attempt in range(1, attempts + 1):
        try:
            return await unstable_request()
        except ValueError as e:
            print(f"Attempt {attempt}: {e}")
            if attempt == attempts:
                print("Unstable request failed after 3 attempts.")
                return None
        await asyncio.sleep(1)

async def api_aggregator():
    tasks = [
        asyncio.create_task(fast_request()),
        asyncio.create_task(slow_request()),
        asyncio.create_task(handle_unstable_request()),
    ]


    for task in asyncio.as_completed(tasks):
        try:
            result = await task
            if result is not None:
                print(f"Result: {result}")
        except Exception as e:
            print(f"Error: {e}")

await api_aggregator()

Result: Fast result
Result: Unstable result
Result: Slow result
